<a href="https://colab.research.google.com/github/zuruoke/hello-world/blob/master/CIfar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2


In [0]:
NUM_CLASSES = 10
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
VALIDATION_SPLIT = 0.15
EPOCHS = 70
BATCH_SIZE = 128
VERBOSE=1
OPTIM = tf.keras.optimizers.RMSprop()

In [0]:
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    mean = np.mean(x_train, axis=(0,1,2,3))
    std = np.std(x_train, axis=(0,1,2,3))
    x_train = (x_train - mean) / (std+1e-7)
    x_test = (x_test - mean) / (std+1e-7)
    
    x_train = x_train.reshape((50000, 32, 32, 3))
    x_test = x_test.reshape((10000, 32, 32, 3))
    
    y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test =  tf.keras.utils.to_categorical(y_test, NUM_CLASSES)
    return x_train, y_train, x_test, y_test 

In [0]:
(x_train, y_train, x_test, y_test ) = load_data()

In [0]:
def build_model():
    # first block
    model = models.Sequential()
    model.add(layers.Convolution2D(32, (3,3),input_shape=(32,32, 3),padding="same",activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(32, (3,3), padding="same", activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Dropout(0.2))
    
    #second block
    model.add(layers.Convolution2D(64, (3,3),padding="same", activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(64, (3,3),padding="same", activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Dropout(0.3))
    
    #third block
    model.add(layers.Convolution2D(128, (3,3),padding="same", activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(128, (3,3),padding="same",activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(128, (3,3), padding="same", activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Dropout(0.5))
    
    
    #fourth block
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation="relu"))
    model.add(layers.Dense(500, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation="softmax"))
    return model

In [0]:
model = build_model()
model.compile(
              loss="categorical_crossentropy",
              optimizer='sgd',
              metrics=["accuracy"]
)

In [134]:
model.fit(x_train, y_train, epochs=200, batch_size=80, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)

Train on 42500 samples, validate on 7500 samples
Epoch 1/200
42500/42500 [==============================] - 7s 155us/sample - loss: 1.9481 - accuracy: 0.3064 - val_loss: 1.7325 - val_accuracy: 0.3799
Epoch 2/200
42500/42500 [==============================] - 6s 133us/sample - loss: 1.5601 - accuracy: 0.4218 - val_loss: 1.5064 - val_accuracy: 0.4543
Epoch 3/200
42500/42500 [==============================] - 6s 132us/sample - loss: 1.4153 - accuracy: 0.4798 - val_loss: 1.3730 - val_accuracy: 0.5121
Epoch 4/200
42500/42500 [==============================] - 6s 132us/sample - loss: 1.3064 - accuracy: 0.5201 - val_loss: 1.2548 - val_accuracy: 0.5411
Epoch 5/200
42500/42500 [==============================] - 6s 131us/sample - loss: 1.2033 - accuracy: 0.5621 - val_loss: 1.1025 - val_accuracy: 0.5916
Epoch 6/200
42500/42500 [==============================] - 6s 133us/sample - loss: 1.1331 - accuracy: 0.5910 - val_loss: 1.2554 - val_accuracy: 0.5668
Epoch 7/200
42500/42500 [====================

In [135]:
score = model.evaluate(x_test, y_test)
print(score[0])

10000/10000 [==============================] - 1s 110us/sample - loss: 0.6898 - accuracy: 0.8587
0.6897526976644993
